In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from citipy import citipy
import requests
from datetime import datetime
from config import weather_api_key

In [2]:
# create a set of random lat and long combinations
lats = np.random.uniform(low = -90, high = 90, size = 1500)
lngs = np.random.uniform(low = -180, high = 180, size = 1500)
lat_lngs = zip(lats, lngs)
lat_lngs

In [3]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

In [4]:
# city list
cities = []

# gets nearest city for each lat/long combo
for coords in coordinates:
    city = citipy.nearest_city(coords[0], coords[1]).city_name
    
    # if the city is unique, then add to list
    if city not in cities:
        cities.append(city)

# print count to confirm sufficient
len(cities)

602

In [5]:
# basic url for API
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

In [6]:
# list of city data
city_data = []

# print beginning of the logging
print('Beginning Data Retrieval     ')
print('-----------------------------')

# counters
record_count = 1
set_count = 1

# loop through all cities in list
for i, city in enumerate(cities):
    
    # group cities in sets of 50 for logging purposes
    if(i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    
    # endpoint URL for ech city
    city_url = url + '&q=' + city.replace(' ', '+')
    
    # log the url, record, and set numbers for the city
    print(f'Processing Record {record_count} of Set {set_count} | {city}')
    
    # add 1 to record count
    record_count += 1
    
    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | norman wells
Processing Record 2 of Set 1 | luang prabang
Processing Record 3 of Set 1 | punta arenas
Processing Record 4 of Set 1 | new norfolk
Processing Record 5 of Set 1 | poyarkovo
Processing Record 6 of Set 1 | ilebo
Processing Record 7 of Set 1 | sentyabrskiy
City not found. Skipping...
Processing Record 8 of Set 1 | vaini
Processing Record 9 of Set 1 | faanui
Processing Record 10 of Set 1 | abay
Processing Record 11 of Set 1 | hilo
Processing Record 12 of Set 1 | tuktoyaktuk
Processing Record 13 of Set 1 | amderma
City not found. Skipping...
Processing Record 14 of Set 1 | tsihombe
City not found. Skipping...
Processing Record 15 of Set 1 | avarua
Processing Record 16 of Set 1 | sao joao evangelista
Processing Record 17 of Set 1 | zemlyansk
Processing Record 18 of Set 1 | ushuaia
Processing Record 19 of Set 1 | yar-sale
Processing Record 20 of Set 1 | belushya guba
City not found. Skippin

Processing Record 36 of Set 4 | kuteynykove
Processing Record 37 of Set 4 | umzimvubu
City not found. Skipping...
Processing Record 38 of Set 4 | macas
Processing Record 39 of Set 4 | nome
Processing Record 40 of Set 4 | faya
Processing Record 41 of Set 4 | mys shmidta
City not found. Skipping...
Processing Record 42 of Set 4 | kiunga
Processing Record 43 of Set 4 | kavaratti
Processing Record 44 of Set 4 | nishihara
Processing Record 45 of Set 4 | saleaula
City not found. Skipping...
Processing Record 46 of Set 4 | tampere
Processing Record 47 of Set 4 | weligama
Processing Record 48 of Set 4 | cidreira
Processing Record 49 of Set 4 | saldanha
Processing Record 50 of Set 4 | beringovskiy
Processing Record 1 of Set 5 | bestobe
Processing Record 2 of Set 5 | mgachi
Processing Record 3 of Set 5 | magadan
Processing Record 4 of Set 5 | tarudant
City not found. Skipping...
Processing Record 5 of Set 5 | severo-kurilsk
Processing Record 6 of Set 5 | santa isabel do rio negro
Processing Reco

Processing Record 22 of Set 8 | kalmunai
Processing Record 23 of Set 8 | lavrentiya
Processing Record 24 of Set 8 | provideniya
Processing Record 25 of Set 8 | ranfurly
Processing Record 26 of Set 8 | sola
Processing Record 27 of Set 8 | tarakan
Processing Record 28 of Set 8 | zeya
Processing Record 29 of Set 8 | nipawin
Processing Record 30 of Set 8 | atar
Processing Record 31 of Set 8 | talnakh
Processing Record 32 of Set 8 | merauke
Processing Record 33 of Set 8 | anadyr
Processing Record 34 of Set 8 | jaque
Processing Record 35 of Set 8 | taoudenni
Processing Record 36 of Set 8 | barawe
City not found. Skipping...
Processing Record 37 of Set 8 | karatsu
Processing Record 38 of Set 8 | shelburne
Processing Record 39 of Set 8 | wuwei
Processing Record 40 of Set 8 | ust-nera
Processing Record 41 of Set 8 | sawtell
Processing Record 42 of Set 8 | coulihaut
City not found. Skipping...
Processing Record 43 of Set 8 | canico
Processing Record 44 of Set 8 | tingo maria
Processing Record 45

Processing Record 11 of Set 12 | carndonagh
Processing Record 12 of Set 12 | torbay
Processing Record 13 of Set 12 | kolokani
Processing Record 14 of Set 12 | brae
Processing Record 15 of Set 12 | verkhnevilyuysk
Processing Record 16 of Set 12 | baglan
Processing Record 17 of Set 12 | nemuro
Processing Record 18 of Set 12 | hirado
Processing Record 19 of Set 12 | ust-kuyga
Processing Record 20 of Set 12 | yumen
Processing Record 21 of Set 12 | zhenjiang
Processing Record 22 of Set 12 | prabumulih
Processing Record 23 of Set 12 | misratah
Processing Record 24 of Set 12 | chifeng
Processing Record 25 of Set 12 | sao jose da coroa grande
Processing Record 26 of Set 12 | preobrazheniye
Processing Record 27 of Set 12 | vila
Processing Record 28 of Set 12 | kholodnyy
Processing Record 29 of Set 12 | post falls
Processing Record 30 of Set 12 | sao joao da barra
Processing Record 31 of Set 12 | ruatoria
City not found. Skipping...
Processing Record 32 of Set 12 | santa maria
Processing Record 

In [7]:
len(city_data)

552